In [14]:
#!/usr/bin/env python
# coding: utf-8

# In[25]:


import pandas as pd

# We collect the result after 23 hours
TIME = 82800

# the list of fuzzers included in 22th May, 2023 experiment
fuzzer_list_1 = [
    "mopt",
    "aflsmart",
    "aflfast",
    "aflplusplus",
    "honggfuzz",
    "libfuzzer",
    "libafl",
    "libafl_fuzzbench_cov_accounting",
    "libafl_fuzzbench_explore",
    "libafl_fuzzbench_mopt",
    "libafl_fuzzbench_value_profile",
    "libafl_fuzzbench_weighted",
    "libafl_fuzzbench_cmplog",
    "libafl_fuzzbench_naive",
    "libafl_fuzzbench_fast",
    "libafl_fuzzbench_rand_scheduler",
]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)


df = pd.read_csv("./experiments/libafl0522.csv", engine='python')
selected = df[df['fuzzer'].isin(fuzzer_list_1)]
selected = selected[['fuzzer', 'benchmark', 'edges_covered', 'time']]
selected = selected[selected['time'] == TIME]

benchmarks = selected.benchmark.unique()

# Some experiment was not complete so we ran it again on 2th June, 2023
fuzzer_list_2 = [
    "libafl_fuzzbench_ngram4",
    "libafl_fuzzbench_ngram8",
    "libafl_fuzzbench_naive_ctx",
]

dff = pd.read_csv("./experiments/libafl0602.csv", engine='python')
selectedd = dff[dff['fuzzer'].isin(fuzzer_list_2)]
selectedd = selectedd[['fuzzer', 'benchmark', 'edges_covered', 'time']]
selectedd = selectedd[selectedd['time'] == TIME]

# Lastly we added fix to grimoire fuzzer, so rerun the experiment on 25th September 2023 
fuzzer_list_3 = [
    "libafl_fuzzbench_grimoire",
]
dfff = pd.read_csv("./experiments/libafl0925.csv", engine = 'python')
selecteddd = dfff[dfff['fuzzer'].isin(fuzzer_list_3)]
selecteddd = selecteddd[['fuzzer', 'benchmark', 'edges_covered', 'time']]
selecteddd = selecteddd[selecteddd['time'] == TIME]

# Concat them together
result = pd.concat([selected, selectedd, selecteddd])
fuzzer_list = fuzzer_list_1 + fuzzer_list_2 + fuzzer_list_3

result.to_csv("result.csv")

import matplotlib.pyplot as plt

import json
import numpy as np



In [15]:
# Cohen
# In[ ]:
import matplotlib
corr_result = dict()
import warnings
pd.options.mode.chained_assignment = None 
warnings.filterwarnings("ignore")
from numpy import std, mean, sqrt, nan
import numpy

printf_debug = False

# pooled deviation, this is needed to calculate cohen's d
def pooled_dev(s1, s2):
    l1 = len(s1)
    l2 = len(s2)
    
    return sqrt((numpy.var(s1, ddof = 1) * (l1 - 1) + numpy.var(s2, ddof = 1) * (l2 - 1)) / (l1 + l2 - 2))

# to color the graphs
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

# Cohen's d
def cohen_d(fuzzer, x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    divisor = sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    d = (mean(x) - mean(y)) / divisor

    wa = nx+ny
    seki = nx * ny
    nijo = d * d

    # std error of cohen's d, from
    # https://stats.stackexchange.com/questions/495015/what-is-the-formula-for-the-standard-error-of-cohens-d
    # we didn't use it in the end
    v = sqrt((wa / seki) + (nijo / (2 * (wa - 2))))

    # for debug
    # if numpy.isnan(v) or v == 0:
    #     print(fuzzer, nx, ny, dof)
    #     print(x, y)
    return (d, divisor, v)

plot = False

def run_analysis(abs_path, property_data, filename):
    if printf_debug:
        print("Loading", abs_path)
    res_spearman = dict()
    res_cohen = dict()
    
    for FUZZER in fuzzer_list:
        points_removed = 0
        points = dict()

        property_v = []
        for benchmark in benchmarks:
            property_v.append(property_data[benchmark][1])

        # Find the the ICQ Range
        q75, q25 = np.percentile(property_v, [75 ,25])
        x_iqr = q75 - q25
        x_min = q25 - x_iqr * 1.5
        x_max = q75 + x_iqr * 1.5

        divisor = dict()
        # print(filename, outlier_x, "outliers")
        for benchmark in benchmarks:

            property_rank, property_value = property_data[benchmark]
            # Remove feature outliers
            if property_value < x_min or property_value > x_max:
               continue
            for fuzzer in fuzzer_list:
                if fuzzer == FUZZER:
                    data = result[(result['benchmark'] == benchmark)]
                    property_rank, property_value = property_data[benchmark]

                    # Find out the rank
                    data.loc[:, 'fuzzer_rank'] = data.loc[:, 'edges_covered'].rank(method = 'average')
                    # This is the baseline data
                    baseline = data[(data['fuzzer'] == 'libafl_fuzzbench_naive')]

                    # Set for target fuzzer
                    s1 = []
                    # Set for the baseline (i.e. naive)
                    s2 = []
                    target_fuzzer = data[(data['fuzzer'] == fuzzer)]
                    for v in target_fuzzer['edges_covered']:
                        s1.append(v)
                    for v in baseline['edges_covered']:
                        s2.append(v)
                        
                    if len(s1) != 0 and fuzzer != 'libafl_fuzzbench_naive':
                        cohend, divisor, v = cohen_d(fuzzer, s1, s2)
                        points[benchmark] = (((divisor, cohend - 2*v, cohend, cohend + 2*v), property_value))

        # ss.sort(key = lambda item: item[1])
        X = []
        
        y = []
        if len(points) == 0:
            continue
            
        cmap = get_cmap(len(points.keys()))
        points = {k: v for k, v in sorted(points.items(), key = lambda x: x[1][1])}
        names = []
        for i, (bch_name, ((divisor, l, cohend, h), property_value)) in enumerate(points.items()):
            X.append(property_value)
            y.append(cohend)
            names.append(bch_name)
            plt.scatter([property_value], [cohend], c = cmap(i), label = bch_name)

        if printf_debug:
            print("names", names)
            print("X", X)
            print("y", y)
        reg = np.polyfit(X, y, 1)
        f = np.poly1d(reg)

        # Plot the graph
        plt.xlabel("program feature value")
        plt.ylabel("cohen d")
        filename_toshow = filename.replace("_ord.txt", "")
        fuzzername_toshow = FUZZER.replace("libafl_fuzzbench_", "")
        plt.title("{}-{}".format(filename_toshow, fuzzername_toshow))
        plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
        plt.plot(X, f(X), color = "r")
        # print(f)
        
        from scipy import stats
        import math
        # print(stats.spearmanr(X, y))
        cor_result = stats.pearsonr(X, y)
        spearman_r = cor_result.statistic
        pvalue = cor_result.pvalue
        count = len(X)

        res_spearman[FUZZER] = (spearman_r, pvalue)
        plt.clf()
    
    for (key, (r, pvalue)) in res_spearman.items():
        if (r >= 0.40 or r <= -0.40) and pvalue <= 0.05:
            if key in corr_result:
                corr_result[key].append((file, (r, pvalue)))
            else:
                corr_result[key] = [(file, (r, pvalue))]

# Place the data in '../data' to load
import os
file_list = []
for r, subdir, files in os.walk("../data"):
    for file in files:
        abs_path = os.path.join(r, file)
        file_list.append((abs_path, file))

# we don't really need to sort but just for debug
file_list.sort()
for abs_path, file in file_list:
    property_data = dict()
    with open(abs_path) as f:
        property_data = json.load(f)
    assert(len(property_data) == 23)
    run_analysis(abs_path, property_data, file)
# In[ ]:

available_features = dict()
for k, v in corr_result.items():
    if printf_debug:
        print("Result: ")
        print("key", k)
        print("value", v)
    available_features[k] = []
    for item in v:
        available_features[k].append(item[0].lstrip("../data/"))

with open("cohen_features.json", 'w') as json_output:
    json.dump(available_features, json_output)

print("Dumped the correlated features to ./cohen_features.json !")
# print(corr_result)



Dumped the correlated features to ./cohen_features.json !


<Figure size 640x480 with 0 Axes>